<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/Scraping/3_Selenium_y_xpath/ejercicio/linkedin_solucion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" data-canonical-src="https://colab.research.google.com/assets/colab-badge.svg"></a>

In [6]:
# !pip install webdriver_manager

In [6]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

class Scraper:
    """Clase base para scrapear basada en selenium"""

    def __init__(self, user:str, passw:str):
        self.user = user
        self.passw = passw

    def log_in(self):
        pass

    def upload(self):
        pass

    @staticmethod
    def init_chrome(
        headless=False,
        no_images=False,
        datadir=None,
        chrome_path=None,
        mobile=False,
    ):

        chrome_options = Options()
        chrome_options.add_argument("--headless") if headless else None
        chrome_options.add_argument(f"user-data-dir={datadir}") if datadir else None
        chrome_options.add_argument("start-maximized")
        chrome_options.add_experimental_option(
            "prefs", {"profile.managed_default_content_settings.images": 2}
        ) if no_images else None

        if mobile:
            chrome_options.add_experimental_option(
                "mobileEmulation",
                {
                    "deviceMetrics": {"width": 360, "height": 640, "pixelRatio": 3.0},
                    "userAgent": (
                        "Mozilla/5.0 (Linux; Android 9; Redmi Note 7) "
                        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 "
                        "Mobile Safari/537.36"
                    ),
                    "touch": True,
                },
            )
            chrome_options.add_experimental_option("w3c", False)

            useragent = (
                "Mozilla/5.0 (Linux; Android 9; Redmi Note 7) AppleWebKit/537.36 "
                "(KHTML, like Gecko) Chrome/78.0.3904.108 Mobile Safari/537.36"
            )
        else:
            useragent = (
                "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like "
                "Gecko) Chrome/73.0.3683.103 Safari/537.36"
            )
            chrome_options.add_argument(
                f"user-agent={useragent}"
            ) if useragent else None

        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-gpu")

        driver = webdriver.Chrome(
            executable_path=ChromeDriverManager().install(),
            options=chrome_options,
            desired_capabilities=chrome_options.to_capabilities(),
        )

        # driver.set_window_size(windowsize[0], windowsize[1]) if windowsize else None
        print("Running Chrome.")
        return driver


class Linkedin(Scraper):
    def __init__(self, headless=True, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.driver = self.init_chrome(headless=headless)
        self.base_url = "https://www.linkedin.com"

    def log_in(self):
        # Ingresa usuario
        self.driver.get("https://www.linkedin.com/login?")
        el = self.driver.find_element(By.XPATH, value="session_key")
        el.send_keys(self.user)

        # Ingresa password
        password = self.driver.find_element(By.XPATH, value="session_password")
        password.send_keys(self.passw)

        # Click boton loggin
        boton_log_in = self.driver.find_element(By.CLASS_NAME, value="btn__primary--large")
        boton_log_in.click()

    def upload(self, content):
        # Click boton iniciar posteo
        boton_post = self.driver.find_element(By.XPATH, value="//button/span[text()='Start a post']")
        boton_post.click()

        # Contenido post
        contenido_post = self.driver.find_element(By.XPATH, value="//div[@data-placeholder='What do you want to talk about?']"
        )
        contenido_post.send_keys(content)

        # Boton postear
        boton_post2 = self.driver.find_element(By.XPATH, value="//span[text()='Post']")
        boton_post2.click()

In [3]:
from getpass import getpass

lk = Linkedin(headless=False, user='mtsgrinberg@gmail.com', passw=getpass())

 ········




====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST chromedriver version for 94.0.4606 google-chrome
Driver [/home/cerebrock/.wdm/drivers/chromedriver/linux64/94.0.4606.113/chromedriver] found in cache
<ipython-input-2-1a8b7d2fbc16>:67: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


Running Chrome.


In [ ]:
lk.log_in()

In [ ]:
lk.upload('Post hecho desde Python con Selenium!')